In [4]:
import pandas as pd
import config
from binance.client import Client
from modules import *

In [5]:
client=Client(config.api_key,config.secret_key)

In [11]:
exchange_info = client.futures_exchange_info()

for symbol in exchange_info['symbols']:
    if symbol['symbol'] == f"{coin}USDT":
        round_quantity = symbol['quantityPrecision']
        break

In [13]:
round_quantity

3

In [114]:
coin = 'ETH'
entry = 1846.56

round_price = 2
take_profit = 1900
lowerband = 1700
upperband = 1800

quantity = round(24/entry, round_quantity)

In [117]:
o = Order(coin = coin,
                            entry = entry,
                            quantity = quantity,
                            round_price = round_price,
                            change = None,
                            take_profit  = take_profit,
                            lowerband = lowerband,
                            upperband = upperband
                            )

In [106]:
o

In [107]:
order_details = client.futures_create_order(symbol=f'{o.coin}USDT', side='BUY', type='MARKET', quantity=o.quantity, dualSidePosition=True, positionSide='LONG')


In [108]:
order_details

{'orderId': 8389765622980676606,
 'symbol': 'ETHUSDT',
 'status': 'NEW',
 'clientOrderId': 'XiLAaGp82KqQpGNE7S0SBx',
 'price': '0.00',
 'avgPrice': '0.00',
 'origQty': '0.014',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'BUY',
 'positionSide': 'LONG',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'MARKET',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1698148755649}

In [119]:
limit_order_details =  client.futures_create_order(
                                    symbol=f'{o.coin}USDT',
                                    price=round(o.take_profit,o.round_price),
                                    side='SELL',
                                    positionSide='LONG',
                                    quantity=o.take_profit,
                                    timeInForce='GTC',
                                    type='LIMIT',
                                    # reduceOnly=True,cc
                                    closePosition=False,
                                    # stopPrice=round(take_profit,2),
                                    workingType='MARK_PRICE',
                                    priceProtect=True
                                )

In [120]:
limit_order_details

{'orderId': 8389765622982524493,
 'symbol': 'ETHUSDT',
 'status': 'NEW',
 'clientOrderId': 'r10TXVydtzXHpEnOJmD5aA',
 'price': '1900.00',
 'avgPrice': '0.00',
 'origQty': '0.014',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'reduceOnly': True,
 'closePosition': False,
 'side': 'SELL',
 'positionSide': 'LONG',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'LIMIT',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1698149041068}

In [118]:
o.entry 

1846.56

In [123]:
price = 1851.43


rounded_price = round(price, o.round_price)
rounded_price

1851.43

In [124]:
dca_order_details =   client.futures_create_order(
            symbol=f'{o.coin}USDT',
            side='BUY',
            type='LIMIT',
            price=rounded_price,
            quantity=o.quantity,
            timeInForce='GTC',
            positionSide='LONG',
            dualSidePosition=True
        )


In [125]:
dca_order_details

{'orderId': 8389765622983006486,
 'symbol': 'ETHUSDT',
 'status': 'NEW',
 'clientOrderId': 't2w43jU1hPC5wRGFSTeoWE',
 'price': '1851.43',
 'avgPrice': '0.00',
 'origQty': '0.013',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'BUY',
 'positionSide': 'LONG',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'LIMIT',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1698149108369}

{8389765622804036829: 8389765622804084388}

In [68]:
master_order_history = {}

In [69]:
coin_dict = master_order_history.setdefault(o.coin, {})
buy_dict = coin_dict.setdefault('Buy', {})

buy_dict[o.take_profit] = {limit_order_details['orderId'] : dca_order_details['orderId']}

In [71]:
buy_dict[1820] = {limit_order_details['orderId']+1 : dca_order_details['orderId']+1}

In [72]:
master_order_history

{'ETH': {'Buy': {1800: {8389765622804036829: 8389765622804084388},
   1820: {8389765622804036830: 8389765622804084389}}}}

In [81]:
high_price = 1771.5
dca_orders = []

# Checking if the coin and 'Buy' dictionary exist
exit_outer_loop = False

while True:
    high_price += 5
    print(high_price)
    if o.coin in master_order_history and 'Buy' in master_order_history[o.coin]:
        buy_dict = master_order_history[o.coin]['Buy']
        for rounded_price in list(buy_dict.keys()):  # iterate over a copy of the keys
            if high_price > rounded_price:
                dca_orders.extend(buy_dict[rounded_price].values())
                del buy_dict[rounded_price]  # remove the key-value pair from the dictionary
                exit_outer_loop = True
                break
                
    if exit_outer_loop:
        break

print(dca_orders)


1776.5
1781.5
1786.5
1791.5
1796.5
1801.5
1806.5
1811.5
1816.5
1821.5
[8389765622804084389]


In [82]:
master_order_history

{'ETH': {'Buy': {}}}

In [ ]:

Buy : {


   'limitOrderPirce' :  'orderID' 
}

sell : {

    'limitOrderPirce' :  'orderID' 
}

if currentPrice > 'limitOrderPirce':
    cancel that limit order


In [83]:
master_order_history = {
    'BTC': {  # Bitcoin
        'Sell': {
            45000: {  # Price at which the sell order is set
                'limit_order_001': 'dca_order_001',
                'limit_order_002': 'dca_order_002'
            },
            46000: {
                'limit_order_003': 'dca_order_003',
            }
        },
        'Buy': {
            44000: {
                'limit_order_004': 'dca_order_004',
            }
        }
    }
}



In [93]:
dca_orders = []
master_order_history

{'BTC': {'Sell': {45000: {'limit_order_001': 'dca_order_001',
    'limit_order_002': 'dca_order_002'},
   46000: {'limit_order_003': 'dca_order_003'}},
  'Buy': {44000: {'limit_order_004': 'dca_order_004'}}}}

In [94]:
low = 45999

In [95]:
if 'BTC' in master_order_history and 'Sell' in master_order_history['BTC']:
    buy_dict = master_order_history['BTC']['Sell']
    for rounded_price in list(buy_dict.keys()):  # iterate over a copy of the keys
        if low < rounded_price:
            dca_orders.extend(buy_dict[rounded_price].values())

In [96]:
dca_orders

['dca_order_003']

In [98]:
buy_dict = master_order_history['BTC']['Sell']
buy_dict

{45000: {'limit_order_001': 'dca_order_001',
  'limit_order_002': 'dca_order_002'},
 46000: {'limit_order_003': 'dca_order_003'}}

In [99]:
list(buy_dict.keys())

[45000, 46000]

In [102]:
buy_dict[45000]

{'limit_order_001': 'dca_order_001', 'limit_order_002': 'dca_order_002'}

In [126]:
account_history = client.futures_account_trades(limit = 1000)
df = pd.DataFrame(account_history)

In [127]:
df

,symbol,id,orderId,side,price,qty,realizedPnl,marginAsset,quoteQty,commission,commissionAsset,time,positionSide,buyer,maker
0,BNXUSDT,40777152,1236281070,SELL,0.279200,146.7,0,USDT,40.9586400,0.02047931,USDT,1697545422867,SHORT,False,False
1,BNXUSDT,40777153,1236281070,SELL,0.279100,678.4,0,USDT,189.3414400,0.09467072,USDT,1697545422867,SHORT,False,False
2,BNXUSDT,40777154,1236281070,SELL,0.279100,943.3,0,USDT,263.2750300,0.13163751,USDT,1697545422867,SHORT,False,False
3,BNXUSDT,40777155,1236281070,SELL,0.279100,142.1,0,USDT,39.6601100,0.01983005,USDT,1697545422867,SHORT,False,False
4,BNXUSDT,40777156,1236281070,SELL,0.279100,300,0,USDT,83.7300000,0.04186500,USDT,1697545422867,SHORT,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,UNFIUSDT,280835705,4651760058,BUY,7.331,3.3,0,USDT,24.1923,0.01209615,USDT,1698114546678,LONG,True,False
888,UNFIUSDT,280838492,4651760267,SELL,7.399,1,0.06800000,USDT,7.3990,0.00147980,USDT,1698114831417,LONG,False,True
889,UNFIUSDT,280838493,4651760267,SELL,7.399,2.3,0.15640000,USDT,17.0177,0.00340354,USDT,1698114831438,LONG,False,True
890,ETHUSDT,3300807759,8389765622980676606,BUY,1842.13,0.014,0,USDT,25.78982,0.01289491,USDT,1698148755649,LONG,True,False


In [130]:
exchange_info = client.futures_exchange_info()
o.coin = 'MINA'
coin = o.coin
for symbol in exchange_info['symbols']:
    if symbol['symbol'] == f"{coin}USDT":
        round_quantity = symbol['quantityPrecision']
        print(f'Precesion for coin : {coin} : {round_quantity}')
        break

Precesion for coin : MINA : 0


In [132]:
limit_order_details =  client.futures_create_order(
                                    symbol=f'{o.coin}USDT',
                                    price=0.9000,
                                    side='SELL',
                                    positionSide='LONG',
                                    quantity=100,
                                    timeInForce='GTC',
                                    type='LIMIT',
                                    # reduceOnly=True,cc
                                    closePosition=False,
                                    # stopPrice=round(take_profit,2),
                                    workingType='MARK_PRICE',
                                    priceProtect=True
                                )